- Inspired from demo.ipynb in NGSIM.jl
- Takeaways
    - 101 has 5 segments
    - Start from bottom of screen, 1 is bottom of screen, 2 is exit ramp,
    3 is straight section, 4 is entry to road till entry ramp 
    join point, 5 is entry ramp

- For `observe!` from `Tim2DDriver` to work, had to make the following changes
    - Had to modify `AutomotiveDrivingModels.jl/src/2d/vehicles/scene_records.jl` to extend
the default length of the capacity in the record container from 100 to 300
    - Did the same capacity extension in `Records.jl/src/frames.jl`, `QueueRecords.jl` and 
`ListRecords.jl`
- Note that querying Tim2DDriver yields a `latLonAccel` return type. Had to create an array
to enable these values to work with `step`

In [1]:
using NGSIM
using AutomotiveDrivingModels
using AutoViz
using Interact # For slider bar trajectory propagation
using Reel

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/5dc1e185f8b2175037a0bb4bac5baa1c8ca39ea5-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
roadway = ROADWAY_101;

In [3]:
using AutoEnvs

n_veh = 1
filepath = Pkg.dir("NGSIM", "data", 
    "trajdata_i101_trajectories-0805am-0820am.txt")
params = Dict(
        "trajectory_filepaths"=>[filepath],
        "H"=>200,
        "primesteps"=>50,
        "n_veh"=>n_veh,
        "remove_ngsim_veh"=>false
)
# env = MultiagentNGSIMEnvVideoMaker(params)
env = MultiagentNGSIMEnv(params);
timestep = 0.1;

In [ ]:
model = Tim2DDriver(timestep,mlane = MOBIL(timestep));

In [7]:
mlon = IntelligentDriverModel(σ=0.1,k_spd=1.0,T=0.1,s_min=2.0,
    a_max=3.0,d_cmf=2.5)
mlat = ProportionalLaneTracker(σ=0.1, kp=3.0, kd=2.0)
mlane = MOBIL(timestep,politeness=0.1,advantage_threshold=0.01)

model = Tim2DDriver(timestep, mlon=mlon, mlat=mlat, mlane=mlane);

In [8]:
# This will create a series of images in /tmp/episode1
# You can open the first and then press right arrow

# a = ones(n_veh, 2)
# a[2] = 0.0
reset(env,random_seed=4);
t_init = env.t
for _ in 1:200
    render(env)
    observe!(model, env.scene, env.roadway, env.ego_vehs[1].id)
    latlonacc = rand(model)
    a = [latlonacc.a_lon latlonacc.a_lat]
    @show env.t-t_init a
    
    # Don't we need accelturnrate for the env?
    x, r, terminal, _ = step(env, a)
end

env.t - t_init = 0
a = [2.43104 -0.0123736]
env.t - t_init = 1
a = [2.39095 0.362963]
env.t - t_init = 2
a = [2.39237 -0.24747]
env.t - t_init = 3
a = [2.12051 0.385528]
env.t - t_init = 4
a = [2.04167 -0.293253]
env.t - t_init = 5
a = [1.82518 -0.160049]
env.t - t_init = 6
a = [1.76268 0.0920505]
env.t - t_init = 7
a = [1.67108 0.23403]
env.t - t_init = 8
a = [1.17765 -0.211872]
env.t - t_init = 9
a = [1.20095 0.188735]
env.t - t_init = 10
a = [0.890146 -0.338252]
env.t - t_init = 11
a = [0.56142 0.353502]
env.t - t_init = 12
a = [0.348597 -0.208093]
env.t - t_init = 13
a = [0.254134 -0.0998311]
env.t - t_init = 14
a = [0.172692 0.272058]
env.t - t_init = 15
a = [-0.138249 -0.330576]
env.t - t_init = 16
a = [-0.142003 0.269609]
env.t - t_init = 17
a = [-0.136835 0.0192627]
env.t - t_init = 18
a = [-0.13808 -0.178425]
env.t - t_init = 19
a = [0.0429861 0.0747291]
env.t - t_init = 20
a = [0.0143671 0.108373]
env.t - t_init = 21
a = [0.457725 -0.0692657]
env.t - t_init = 22
a = [0.651381

LoadError: [91mAssertionError: !(isnan(headway)) && headway > 0[39m

In [ ]:
function makevid(t,dt)
    for _ in t:300
    render(env)
    observe!(model, env.scene, env.roadway, env.ego_vehs[1].id)
    latlonacc = rand(model)
    a = [latlonacc.a_lon latlonacc.a_lat]
    x, r, terminal, _ = step(env, a)
    end
end

In [ ]:
reset(env,random_seed=4)
film = roll(makevid, fps=10, duration=1.0)

In [ ]:
write("mygif.gif",film)

In [ ]:
# Take a look at sisl/gail-driver/julia/envs/Auto2D.jl function reel_drive
# for video making inspiration

## Video making using recorded cars on ngsim
- Trajdatas is what the car trajectories are stored in
- We want to color the ego vehicle differently to be able to see it

In [ ]:
# argument 1 loads i101 7:50 to 8:05.
# load_trajdata function defined in NGSIM.jl/src/trajdata.jl
td1 = load_trajdata(1); 

scene = Scene(500)
egoid = 546

# Drive here in the notebook. Replay the trajectory as recorded in the ngsim data
@manipulate for i in 1000:2000
    temp_scene = get!(scene,td1,i)
    
    carcolors = Dict{Int,Colorant}()
    for veh in temp_scene
        #@show veh.id
        # if veh id matches the egoid color it blue otherwise green
        carcolors[veh.id] = 
        in(veh.id, egoid) ? colorant"blue" : colorant"green"
    end
    render(temp_scene, ROADWAY_101, 
        cam=CarFollowCamera{Int}(546,5.0),
#         cam=StaticCamera(VecE2(1966400, 570900), 5.0),
#         cam=FitToContentCamera(0.),
        car_colors=carcolors)
end